In [1]:
import requests
import time
import random
import os
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import json

In [4]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "es-ES,es;q=0.9",
    "Cache-Control": "max-age=0",
    "cookie": 'cu=es-es; ajs_anonymous_id=e24007b4-48cf-4707-9efa-c7d2a0c14e6d; _fbp=fb.1.1738339157798.520247286968388148; didomi_token=eyJ1c2VyX2lkIjoiMTk0YmQxNjgtMzcwYi02MzQwLTkwZjctZjU4Mzc4MzYyNmM3IiwiY3JlYXRlZCI6IjIwMjUtMDEtMzFUMTU6NTk6MTYuODQ4WiIsInVwZGF0ZWQiOiIyMDI1LTAxLTMxVDE1OjU5OjIwLjU5MVoiLCJ2ZW5kb3JzIjp7ImVuYWJsZWQiOlsiZ29vZ2xlIiwiYzpnb29nbGVhbmEtNFRYbkppZ1IiLCJjOmFkZXZpbnRhLW1vdG9yLW1vdG9yIiwiYzptZXRhcGxhdGYtTlJlVnBtTFEiLCJjOmNyaXRlby1QNDhlR1QydyIsImM6Z29vZ2xlaXJlLWZmS2FQYVJEIl19LCJwdXJwb3NlcyI6eyJlbmFibGVkIjpbImRldmljZV9jaGFyYWN0ZXJpc3RpY3MiLCJnZW9sb2NhdGlvbl9kYXRhIiwidHJhbnNmZXItdG8tbW90b3IiLCJ1c29kZWxvcy1RcExwTThqVyJdfSwidmVuZG9yc19saSI6eyJlbmFibGVkIjpbImdvb2dsZSJdfSwidmVyc2lvbiI6Mn0=; euconsent-v2=CQMGLkAQMGLkAAHABBENBaFsAP_gAEPgAAiQKxtX_G__bWlr8X73aftkeY1P99h77sQxBhbJE-4FzLvW_JwXx2E5NA36tqIKmRIAu3TBIQNlHJDURVCgaogVrSDMaEyUoTNKJ6BkiFMRI2dYCFxvm4tjeQCY5vr991dx2B-t7dr83dzyy4hHn3a5_2S0WJCdA5-tDfv9bROb-9IOd_x8v4v4_F7pE2_eT1l_tWvp7D9-cts_9XW99_fbff9Pn_-uB_-_X_vf_H37gq-ASYaFRAGWBISEGgYQQIAVBWEBFAgCAABIGiAgBMGBTsDABdYSIAQAoABggBAACDIAEAAAEACEQAQAFAgAAgECgADAAgGAgAIGAAEAFgIBAACA6BimBBAIFgAkZkVCmBCEAkEBLZUIJAECCuEIRZ4BEAiJgoAAAAACkAAQFgsDiSQEqEggC4gmgAAIAEAggAKEEnJgACAM2WoPBk2jK0wDB8wSIaYBkARBGQkGgAAA.f_wACHwAAAAA; _gcl_au=1.1.1003412851.1738339161; segment_ga=GA1.1.791422271.1738339161; kanirante-adv-aauid=7a883939-7e66-4c32-8804-3d7c807589c0; adit-xandr-id=761685332701469910; _lr_env_src_ats=false; sui_1pc=1738339161130135AF0DD180BFA49ED5B8F2F54CD24C4D1AE7B89BEB; __gsas=ID=3602a79559ea2bbd:T=1738339161:RT=1738339161:S=ALNI_MYNf_9-bq_3mvQCyBeoBaw0Woqfhw; ASP.NET_SessionId=td3n2pvybdw1jz1o2hpzw0o4; auth=td3n2pvybdw1jz1o2hpzw0o4; _hjSessionUser_89611=eyJpZCI6ImZmYjFkNDNjLTAxODgtNTdhNi05MDA5LTVhN2JiNTRmNmFmNiIsImNyZWF0ZWQiOjE3MzgzNDEzMDU5MzIsImV4aXN0aW5nIjp0cnVlfQ==; _lr_sampling_rate=100; usunico=01/02/2025:19-0986645; reese84=3:Sbvk4rXEPjL9ckK3eXzRCA==:1bhoOyqskEAK08pInvGpsr6KoMzGmbIQlECZ5u47doRekos1+x/rLUSxbtaXHQ+E8+y+wfix2qB95KZsI0i8eAHrp8DJ7cn5kNOUW/2zm8nvGkRb9KQz+plfKrJIQ8c6UOiffU1EzoVp7nZpUXXk43JXyC+hS3HIwS8/6hzT//DBurGXV+xdCBLWOyJNxuIAea3gITflmv7tta5sJpgE9JZdbUt0+Hf4dmgNbFyg9T1tH87fzqhUk1MnurP8CS97JSUmJXuMTb/nWKyfy90rOTqqd7iKP7fnB/6uPXX8HUnj/lWL/tQjN7GWhry03qW/a2TjKWbBk9WC1qxzf1M8IrzSiNDlDQwII+cKxCfLDxlzL3ALDrCnhHWuBDZErFWSBBE84H1hUqK3hKPYyqH2GBajkIfCHibRc9jBtDPKxt26s7XNUUSSi2PPBlq+KoNGlfQAz0/7FJiBsiUVrFywxQ==:Uawk6iNqhO9nrkOwbfQimPNuOKm1brofmFrznyMYcBQ=; re_uuid=73285749-e5e2-4328-818a-1863c77bcedc; AMCVS_05FF6243578784B37F000101%40AdobeOrg=1; AMCV_05FF6243578784B37F000101%40AdobeOrg=-408604571%7CMCIDTS%7C20122%7CMCMID%7C50316837995770300660582095743361496031%7CMCAAMLH-1739091796%7C6%7CMCAAMB-1739091796%7C6G1ynYcLPuiQxYZrsz_pkqfLG9yMXBpb2zX5dvJdYQJzPXImdj0y%7CMCOPTOUT-1738494196s%7CNONE%7CMCAID%7CNONE%7CvVersion%7C4.6.0; borosTcf=eyJwb2xpY3lWZXJzaW9uIjoyLCJjbXBWZXJzaW9uIjoxLCJwdXJwb3NlIjp7ImNvbnNlbnRzIjp7IjEiOnRydWUsIjIiOnRydWUsIjMiOnRydWUsIjQiOnRydWUsIjUiOnRydWUsIjYiOnRydWUsIjciOnRydWUsIjgiOnRydWUsIjkiOnRydWUsIjEwIjp0cnVlfX0sInNwZWNpYWxGZWF0dXJlcyI6eyIxIjp0cnVlfX0=; segment_ga_MH0GDRSFGC=GS1.1.1738486996.7.0.1738486996.60.0.0; _csrf=SxXMaWEckyoir5zd1mLvhiNWf1RTQeAM08ooNOZXa/YKW63ULP/FIxDnqdm8XYuxy9DDEDRWmKAyroJODjslI7eR81WitLZ+TbJz+qaD7bg=; ab.storage.deviceId.c9f9127d-989c-45fe-96f2-f36434db5025=g%3A2a822b2e-c4d7-40c9-359c-23b7b4a3088b%7Ce%3Aundefined%7Cc%3A1738339157252%7Cl%3A1738486997210; _lr_retry_request=true; ab.storage.sessionId.c9f9127d-989c-45fe-96f2-f36434db5025=g%3Abbb56631-c416-8b76-b1fc-85ab05f1fa01%7Ce%3A1738488797473%7Cc%3A1738486997209%7Cl%3A1738486997473; __rtbh.lid=%7B%22eventType%22%3A%22lid%22%2C%22id%22%3A%22tLrNk5Sy9sMhG1XbzbjN%22%2C%22expiryDate%22%3A%222026-02-02T09%3A03%3A17.777Z%22%7D; cto_bundle=qBEA-F9wZmdSTmZUUVpBcm1wbXh6bkpFOHRjdkY0aU5VNmZ4Y3NFMWxCVG9nU3pZdSUyQkhUR2NOcFc5bXFSd012QzJDY2V0UWJUT3lQME42bm1jNDJzdSUyQnlJYlN4WnJ4ZkljU0NPUVAlMkZUMnlsNnlSVDhkTkFrVno3WFIyclRIblVlM1J6M0pYbHNOU09mViUyQmhCb2VZVDFSS0xCUSUzRCUzRA; __rtbh.uid=%7B%22eventType%22%3A%22uid%22%2C%22id%22%3A%22unknown%22%2C%22expiryDate%22%3A%222026-02-02T09%3A03%3A19.651Z%22%7D; __gads=ID=109f959d72de702f:T=1738339161:RT=1738486998:S=ALNI_MbYXbQHld9Xeq2RgSGGq8iMEjhDHg; __gpi=UID=0000102acbadd9f5:T=1738339161:RT=1738486998:S=ALNI_MYhDQA1l8gbr7ZxE3NWCoyzn3JHtA; __eoi=ID=3d7ca7519b1918d9:T=1738339161:RT=1738486998:S=AA-AfjbXQnfwCBirWxkc7rAbyh2B'.encode('utf-8'),
    "Referer": "https://www.fotocasa.es/es/alquiler/viviendas/murcia-provincia/todas-las-zonas/l",
    "Sec-Ch-Device-Memory": "4",
    "Sec-Ch-Ua": '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    "Sec-Ch-Ua-Full-Version-List": '"Not)A;Brand";v="99.0.0.0", "Google Chrome";v="127.0.6533.120", "Chromium";v="127.0.6533.120"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "X-Height": "9999",  # Establece el valor de height
    "X-Width": "1920",   # Puedes agregar otros parámetros, como width
    "X-Depth": "500",    # Ejemplo de parámetro depth, si es necesario
    "X-Color": "blue",   # Ejemplo de parámetro de color, si lo deseas
    "X-Orientation": "landscape"  # Otro ejemplo, si necesitas un parámetro de orientación
}


In [5]:
# Crear una sesión
def check_error_page(html_page):
    # Patrón para buscar frases comunes en una página de error
    patrones_error = [
        r"sentimos la interrupciã\x93n",  # Detecta el mensaje de "Sentimos la interrupción"
    ]
    
    # Convertir el HTML a minúsculas para hacerlo insensible a mayúsculas
    html_lower = html_page.lower()
    # Buscar cada uno de los patrones de error en el HTML
    for patron in patrones_error:
        if re.search(patron, html_lower):
            return True
    return False

session = requests.Session()
session.headers.update(headers)

# URL base
base_url = "https://www.fotocasa.es/es/alquiler/viviendas/murcia-provincia/todas-las-zonas/"

# Función para verificar si es una página de error
def check_error_page(html_page):
    patrones_error = [r"sentimos la interrupciã\x93n"]
    html_lower = html_page.lower()
    for patron in patrones_error:
        if re.search(patron, html_lower):
            return True
    return False

def obtener_tipo_inmueble(buildingType, buildingSubtype):
    tipos = {
        "Flat": "Piso",
        "House": "Casa",
        "Duplex": "Dúplex",
        "Penthouse": "Ático",
        "Studio": "Estudio",
        "Villa": "Villa"
    }
    
    subtipos = {
        "Apartment": "Apartamento",
        "TerracedHouse": "Casa adosada",
        "DetachedHouse": "Casa independiente",
        "SemidetachedHouse": "Casa Semi-Independiente",
        "Loft": "Loft",
        "Farmhouse": "Casa de campo",
        "Townhouse": "Casa de pueblo"
    }

    return subtipos.get(buildingSubtype, tipos.get(buildingType, "Desconocido"))

def obtener_tipo_contacto(clientTypeId):
    tipos_contacto = {
        1: "Particular",
        2: "Promotora",
        3: "Inmobiliaria",
        4: "Banco"
    }
    return tipos_contacto.get(clientTypeId, "Desconocido")

def obtener_estado(coservationSatus):
    estado_conservacion = {
        1: "1: Desconocido",
        2: "Muy Bien",
        3: "Bien",
        4: "4: Desconocido",
        8: "Reformado",
    }
    return estado_conservacion.get(coservationSatus, coservationSatus)

def obtener_antiguedad(antiguedad):
    tipos_antiguedad = {
        1: "Menos de 1 año",
        2: "1 a 5 años",
        3: "5 a 10 años",
        4: "10 a 20 años",
        5: "20 a 30 años",
        6: "30 a 50 años",
    }
    return tipos_antiguedad.get(antiguedad, antiguedad)

def obtener_orientacion(orientacion):
    tipos_orientacion = {
        1: "Norte",
        2: "2: Desconocido",
        3: "3: Desconocido",
        4: "Sur",
        7: "Este"
    }
    return tipos_orientacion.get(orientacion, orientacion)

# Función para extraer los datos de un inmueble según JSON
def extract_property_info(data):
    try:
        traducciones = {
            "parking": "Plaza de aparcamiento",
            "elevator": "Ascensor",
            "furnished": "Amueblado",
            "washing_machine": "Lavadora",
            "fridge": "Frigorífico",
            "bathrooms": "Baños",
            "floor": "Planta",
            "rooms": "Habitaciones",
            "surface": "Superficie",
            "balcony": "Balcón",
            "air_conditioning": "Aire acondicionado",  
            "heating": "Calefacción",  
            "wardrobes": "Armarios empotrados",  
            "garden": "Jardín privado",  
            "pool": "Piscina",
            "terrace": "Terraza",  
            "storage_room": "Trastero",  
            "community_zone": "Zona comunitaria",  
            "kitchen_office": "Cocina Office",  
            "patio": "Patio",  
            "suite_bathroom": "Suite con baño",  
            "appliances": "Electrodomésticos",  
            "oven": "Horno",  
            "microwave": "Microondas",  
            "tv": "TV",  
            "armored_door": "Puerta blindada",  
            "community_pool": "Piscina comunitaria",  
            "laundry": "Lavadero",
            "pets_allowed": "Mascotas Permitidas",
            "community_expenses_included": "Gastos de comunidad incluidos",
            "household_appliances": "Electrodomésticos",
            "yard": "Patio",
            "private_garden": "Jardín Privado",
            "swimming_pool": "Piscina"
        }

        # Función para obtener un valor de las características
        def obtener_caracteristica(key, default=None):
            return next((f['value'] for f in data.get("features", []) if f['key'] == key), default)

        # Extraer todas las características que no sean None
        caracteristicas = [
            f"{traducciones.get(f['key'], f['key'].replace('_', ' ').capitalize())}: {f['value']}"
            for f in data.get("features", [])
            if f['value'] is not None and f['key'] not in [
                'floor', 
                'surface', 
                'elevator', 
                'furnished',
                'rooms',
                'wardrobes',
                'bathrooms',
                'conservationStatus',
                'antiquity',
                'orientation'
                ]
        ]

        # Extraer información relevante
        tipo_inmueble = obtener_tipo_inmueble(data.get("buildingType"), data.get("buildingSubtype"))
        calle = data.get('location', '') or ''
        precio = int(data.get("rawPrice", 0) or 0)
        superficie = obtener_caracteristica("surface")
        precio_metro = ''
        if (precio and superficie):
            precio_metro = precio / superficie
        inmueble = {
            "id_inmueble": data.get("id"),
            "id_fotocasa": data.get("id"),
            "tipo": tipo_inmueble,
            "tipo_transaccion": "Alquiler" if data.get("transactionTypeId") == 3 else "Venta",
            "titulo": f"{tipo_inmueble} en {calle}",
            "calle":  calle,
            "barrio": data["address"].get("neighborhood") or '',
            "zona": data["address"].get("district") or '',
            "ciudad": data["address"].get("city") or '',
            "localizacion": ", ".join(filter(None, [
                data["address"].get("neighborhood") or '',
                data["address"].get("district") or '',
                data["address"].get("city") or '',
                data["address"].get("county") or '',
                data["address"].get("regionLevel1") or ''
            ])),
            "precio": precio,
            "precio_metro": precio_metro,  # No disponible en el JSON
            "caracteristicas": caracteristicas,
            "fecha": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "fuente": "fotocasa",
            "disponibilidad": "disponible",
            "superficie": superficie,
            "habitaciones": obtener_caracteristica("rooms"),
            "banos": obtener_caracteristica("bathrooms"),
            "estado": obtener_estado(obtener_caracteristica('conservationStatus')),
            "armarios": "Armarios empotrados" if obtener_caracteristica("wardrobes") else None,
            "trastero": None,
            "orientacion": obtener_orientacion(obtener_caracteristica('orientation')),
            "calefaccion": None,
            "planta": str(obtener_caracteristica("floor", None)),
            "ascensor": "Con ascensor" if obtener_caracteristica("elevator") else None,
            "construccion": obtener_antiguedad(obtener_caracteristica('antiquity')),
            "amueblado": "Amueblado" if obtener_caracteristica("furnished") else None,
            "movilidad_reducida": None,
            "tipo_contacto": obtener_tipo_contacto(data.get("clientTypeId")),
            "nombre_contacto": data.get("clientAlias", 'No indicado'),
            "telefono_contacto": data.get("phone"),
            "enlace": f"https://www.fotocasa.es{data.get('detail', {}).get('es-ES', '')}"
        }
        return inmueble
    except Exception as e:
        print(f"Error extrayendo datos del inmueble: {e}")
        return None

# Función para manejar diferentes selectores según la estructura de la página
def extract_data_with_selectors(html, json_data=None):
    inmuebles = []

    # Intentar extraer desde JSON
    if json_data:
        try:
            real_estates = json_data['initialSearch']['result']['realEstates']
            for inmueble in real_estates:
                info = extract_property_info(inmueble)
                if info:
                    inmuebles.append(info)
        except KeyError:
            print("El JSON no contiene la clave esperada para 'realEstates'.")

    return inmuebles

# Paginación y scraping
page = 1
all_listings = []
page_limit = 2

while page < page_limit:
    try:
        print(f"Procesando página {page}...")
        response = session.get(f"{base_url}l/{page}?sortType=publicationDate")
        response.raise_for_status()
        error_page = check_error_page(response.text)
        if error_page:
            print("Bloqueado por Fotocasa, cambie la cookie")
            break

        # Intentar extraer JSON
        json_data = None
        match = re.search(
            r'window\.__INITIAL_PROPS__ = JSON\.parse\("((?:[^"\\]|\\.)*)"\);',
            response.text
        )
        if match:
            escaped_json = match.group(1)
            try:
                json_data = json.loads(escaped_json.encode('utf-8').decode('unicode_escape'))
            except json.JSONDecodeError:
                print("Error al decodificar el JSON")

        # Extraer datos usando múltiples estrategias
        inmuebles = extract_data_with_selectors(response.text, json_data)
        all_listings.extend(inmuebles)

        # Pasar a la siguiente página
        page += 1
        time.sleep(random.uniform(5, 10))  # Espera aleatoria para evitar bloqueos

    except Exception as e:
        print(f"Error procesando la página {page}: {e}")
        break

# Convertir a DataFrame
df = pd.DataFrame(all_listings)
df

Procesando página 1...
Bloqueado por Fotocasa, cambie la cookie


""


In [1]:
df

NameError: name 'df' is not defined

In [28]:
df.to_csv('alquileres_fotocasa.csv')

In [21]:
import requests
import re
import json
import time
import random
from datetime import datetime

headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "es-ES,es;q=0.9",
    "Cache-Control": "max-age=0",
    "cookie": 'cu=es-es; ajs_anonymous_id=e24007b4-48cf-4707-9efa-c7d2a0c14e6d; _fbp=fb.1.1738339157798.520247286968388148; didomi_token=eyJ1c2VyX2lkIjoiMTk0YmQxNjgtMzcwYi02MzQwLTkwZjctZjU4Mzc4MzYyNmM3IiwiY3JlYXRlZCI6IjIwMjUtMDEtMzFUMTU6NTk6MTYuODQ4WiIsInVwZGF0ZWQiOiIyMDI1LTAxLTMxVDE1OjU5OjIwLjU5MVoiLCJ2ZW5kb3JzIjp7ImVuYWJsZWQiOlsiZ29vZ2xlIiwiYzpnb29nbGVhbmEtNFRYbkppZ1IiLCJjOmFkZXZpbnRhLW1vdG9yLW1vdG9yIiwiYzptZXRhcGxhdGYtTlJlVnBtTFEiLCJjOmNyaXRlby1QNDhlR1QydyIsImM6Z29vZ2xlaXJlLWZmS2FQYVJEIl19LCJwdXJwb3NlcyI6eyJlbmFibGVkIjpbImRldmljZV9jaGFyYWN0ZXJpc3RpY3MiLCJnZW9sb2NhdGlvbl9kYXRhIiwidHJhbnNmZXItdG8tbW90b3IiLCJ1c29kZWxvcy1RcExwTThqVyJdfSwidmVuZG9yc19saSI6eyJlbmFibGVkIjpbImdvb2dsZSJdfSwidmVyc2lvbiI6Mn0=; euconsent-v2=CQMGLkAQMGLkAAHABBENBaFsAP_gAEPgAAiQKxtX_G__bWlr8X73aftkeY1P99h77sQxBhbJE-4FzLvW_JwXx2E5NA36tqIKmRIAu3TBIQNlHJDURVCgaogVrSDMaEyUoTNKJ6BkiFMRI2dYCFxvm4tjeQCY5vr991dx2B-t7dr83dzyy4hHn3a5_2S0WJCdA5-tDfv9bROb-9IOd_x8v4v4_F7pE2_eT1l_tWvp7D9-cts_9XW99_fbff9Pn_-uB_-_X_vf_H37gq-ASYaFRAGWBISEGgYQQIAVBWEBFAgCAABIGiAgBMGBTsDABdYSIAQAoABggBAACDIAEAAAEACEQAQAFAgAAgECgADAAgGAgAIGAAEAFgIBAACA6BimBBAIFgAkZkVCmBCEAkEBLZUIJAECCuEIRZ4BEAiJgoAAAAACkAAQFgsDiSQEqEggC4gmgAAIAEAggAKEEnJgACAM2WoPBk2jK0wDB8wSIaYBkARBGQkGgAAA.f_wACHwAAAAA; _gcl_au=1.1.1003412851.1738339161; segment_ga=GA1.1.791422271.1738339161; kanirante-adv-aauid=7a883939-7e66-4c32-8804-3d7c807589c0; adit-xandr-id=761685332701469910; _lr_env_src_ats=false; sui_1pc=1738339161130135AF0DD180BFA49ED5B8F2F54CD24C4D1AE7B89BEB; __gsas=ID=3602a79559ea2bbd:T=1738339161:RT=1738339161:S=ALNI_MYNf_9-bq_3mvQCyBeoBaw0Woqfhw; ASP.NET_SessionId=td3n2pvybdw1jz1o2hpzw0o4; auth=td3n2pvybdw1jz1o2hpzw0o4; _hjSessionUser_89611=eyJpZCI6ImZmYjFkNDNjLTAxODgtNTdhNi05MDA5LTVhN2JiNTRmNmFmNiIsImNyZWF0ZWQiOjE3MzgzNDEzMDU5MzIsImV4aXN0aW5nIjp0cnVlfQ==; _lr_sampling_rate=100; usunico=01/02/2025:19-0986645; AMCVS_05FF6243578784B37F000101%40AdobeOrg=1; AMCV_05FF6243578784B37F000101%40AdobeOrg=-408604571%7CMCIDTS%7C20122%7CMCMID%7C50316837995770300660582095743361496031%7CMCAAMLH-1739091796%7C6%7CMCAAMB-1739091796%7C6G1ynYcLPuiQxYZrsz_pkqfLG9yMXBpb2zX5dvJdYQJzPXImdj0y%7CMCOPTOUT-1738494196s%7CNONE%7CMCAID%7CNONE%7CvVersion%7C4.6.0; borosTcf=eyJwb2xpY3lWZXJzaW9uIjoyLCJjbXBWZXJzaW9uIjoxLCJwdXJwb3NlIjp7ImNvbnNlbnRzIjp7IjEiOnRydWUsIjIiOnRydWUsIjMiOnRydWUsIjQiOnRydWUsIjUiOnRydWUsIjYiOnRydWUsIjciOnRydWUsIjgiOnRydWUsIjkiOnRydWUsIjEwIjp0cnVlfX0sInNwZWNpYWxGZWF0dXJlcyI6eyIxIjp0cnVlfX0=; _csrf=SxXMaWEckyoir5zd1mLvhiNWf1RTQeAM08ooNOZXa/YKW63ULP/FIxDnqdm8XYuxy9DDEDRWmKAyroJODjslI7eR81WitLZ+TbJz+qaD7bg=; ab.storage.deviceId.c9f9127d-989c-45fe-96f2-f36434db5025=g%3A2a822b2e-c4d7-40c9-359c-23b7b4a3088b%7Ce%3Aundefined%7Cc%3A1738339157252%7Cl%3A1738486997210; ShowMap=false; segment_ga_MH0GDRSFGC=GS1.1.1738486996.7.0.1738490675.60.0.0; ab.storage.sessionId.c9f9127d-989c-45fe-96f2-f36434db5025=g%3Abbb56631-c416-8b76-b1fc-85ab05f1fa01%7Ce%3A1738492476973%7Cc%3A1738486997209%7Cl%3A1738490676973; _lr_retry_request=true; __rtbh.lid=%7B%22eventType%22%3A%22lid%22%2C%22id%22%3A%22tLrNk5Sy9sMhG1XbzbjN%22%2C%22expiryDate%22%3A%222026-02-02T10%3A04%3A37.107Z%22%7D; __rtbh.uid=%7B%22eventType%22%3A%22uid%22%2C%22id%22%3A%22unknown%22%2C%22expiryDate%22%3A%222026-02-02T10%3A04%3A37.298Z%22%7D; cto_bundle=CFMgtl9wZmdSTmZUUVpBcm1wbXh6bkpFOHRZR3NOJTJGSjJVeldIUkc5ZTdFNXF4UFgxeG5sJTJGd3VwT0VmNEhVcTZWYmNBbERGU0VrQnZSc2FNS09EUWtQRTVzV1lOY2d4OWI2TjdGSG9lczVVWWV4dkhiQmJRd2RIZkdvNnJtcEVyQTZuS21QUFp3TnNLNFlQcU41dzc4aWlSVGlnJTNEJTNE; __gads=ID=109f959d72de702f:T=1738339161:RT=1738490677:S=ALNI_MbYXbQHld9Xeq2RgSGGq8iMEjhDHg; __gpi=UID=0000102acbadd9f5:T=1738339161:RT=1738490677:S=ALNI_MYhDQA1l8gbr7ZxE3NWCoyzn3JHtA; __eoi=ID=3d7ca7519b1918d9:T=1738339161:RT=1738490677:S=AA-AfjbXQnfwCBirWxkc7rAbyh2B; reese84=3:Tip/jD9WlR4/suE9ZkrvcQ==:LUWHDo4lMOApo8Mv/w3ig+WqV11aC2tsBh6lygRprA/Git++XBZx9s0Pargdx8wOiM6NBWW9CcTFwljhmq9PrTlRgPVTp0BPC0a8CrwcfcKum/QY2oTKpBw3/6qlEiKnQ9zmnv1whmaAOo95n/b1JrnPQl0b8HdKLXlW4QZapF+l/+08EMyQjmnI7YAlejpsnlusPLBgtBhY3bKgj6B4fUQZ0y1PsgUXrteEUm0qhE+DS67qa92D3Q5IzDoXquj9WXR0uB0lbjwnbuj9q8n+QGnmtAZDJS57X+03v30P6OiaKn3+Llc/cg8DFCqmU/1olVBZJKs0WhOS7CPR4OjdgrVASohoZdyxYosFwWvoJfZeRLQZ6a+LRFacJpBjBXqF4Ld09Q7cifpBSPtUD48E40h0fLt/EV1bpsW8FmqsOlDLv8kcHe+84Hp5Gc2Zb8hiSLMbdmRirHeQwk7RnzsA9Q==:YbjZhNhQ+xfxKwCeZgtgVHvDGndZW3gQ1YRoPQk7xj0='.encode('utf-8'),
    "Referer": "https://www.fotocasa.es/es/alquiler/viviendas/murcia-provincia/todas-las-zonas/l",
    "Sec-Ch-Device-Memory": "4",
    "Sec-Ch-Ua": '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    "Sec-Ch-Ua-Full-Version-List": '"Not)A;Brand";v="99.0.0.0", "Google Chrome";v="127.0.6533.120", "Chromium";v="127.0.6533.120"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "X-Height": "9999",  # Establece el valor de height
    "X-Width": "1920",   # Puedes agregar otros parámetros, como width
    "X-Depth": "500",    # Ejemplo de parámetro depth, si es necesario
    "X-Color": "blue",   # Ejemplo de parámetro de color, si lo deseas
    "X-Orientation": "landscape"  # Otro ejemplo, si necesitas un parámetro de orientación
}

# Variable global para la URL base de la API
api_base_url = "http://euspay.com/api/v1/euspay.php"
auth_credentials = ("promurcia", "Pr0Murc14")  # Sustituye por tu usuario y contraseña


# ==========================
# 🔹 FUNCIONES AUXILIARES
# ==========================

def obtener_tipo_inmueble(buildingType, buildingSubtype):
    tipos = {
        "Flat": "Piso",
        "House": "Casa",
        "Duplex": "Dúplex",
        "Penthouse": "Ático",
        "Studio": "Estudio",
        "Villa": "Villa"
    }
    
    subtipos = {
        "Apartment": "Apartamento",
        "TerracedHouse": "Casa adosada",
        "DetachedHouse": "Casa independiente",
        "SemidetachedHouse": "Casa Semi-Independiente",
        "Loft": "Loft",
        "Farmhouse": "Casa de campo",
        "Townhouse": "Casa de pueblo"
    }

    return subtipos.get(buildingSubtype, tipos.get(buildingType, "Desconocido"))

def obtener_tipo_contacto(clientTypeId):
    tipos_contacto = {
        1: "Particular",
        2: "Promotora",
        3: "Inmobiliaria",
        4: "Banco"
    }
    return tipos_contacto.get(clientTypeId, "Desconocido")

def obtener_estado(conservationStatus):
    estados = {
        1: "Desconocido",
        2: "Muy Bien",
        3: "Bien",
        4: "Desconocido",
        8: "Reformado"
    }
    return estados.get(conservationStatus, "Desconocido")

def obtener_antiguedad(antiguedad):
    tipos_antiguedad = {
        1: "Menos de 1 año",
        2: "1 a 5 años",
        3: "5 a 10 años",
        4: "10 a 20 años",
        5: "20 a 30 años",
        6: "30 a 50 años"
    }
    return tipos_antiguedad.get(antiguedad, "Desconocido")

def obtener_orientacion(orientacion):
    tipos_orientacion = {
        1: "Norte",
        4: "Sur",
        7: "Este",
        10: "Oeste"
    }
    return tipos_orientacion.get(orientacion, "Desconocido")

# ==========================
# 🔹 EXTRACCIÓN DE DATOS DEL INMUEBLE
# ==========================

def extract_property_info(data):
    try:
        traducciones = {
            "parking": "Plaza de aparcamiento",
            "elevator": "Ascensor",
            "furnished": "Amueblado",
            "washing_machine": "Lavadora",
            "fridge": "Frigorífico",
            "bathrooms": "Baños",
            "floor": "Planta",
            "rooms": "Habitaciones",
            "surface": "Superficie",
            "balcony": "Balcón",
            "air_conditioning": "Aire acondicionado",
            "air_conditioner": "Aire acondicionado",
            "heating": "Calefacción",
            "wardrobes": "Armarios empotrados",
            "garden": "Jardín privado",
            "pool": "Piscina",
            "terrace": "Terraza",
            "storage_room": "Trastero",
            "community_zone": "Zona comunitaria",
            "kitchen_office": "Cocina Office",
            "patio": "Patio",
            "suite_bathroom": "Suite con baño",
            "appliances": "Electrodomésticos",
            "oven": "Horno",
            "microwave": "Microondas",
            "tv": "TV",
            "armored_door": "Puerta blindada",
            "community_pool": "Piscina comunitaria",
            "laundry": "Lavadero",
            "pets_allowed": "Mascotas Permitidas",
            "community_expenses_included": "Gastos de comunidad incluidos",
            "household_appliances": "Electrodomésticos",
            "yard": "Patio",
            "private_garden": "Jardín Privado",
            "swimming_pool": "Piscina"
        }

        def obtener_caracteristica(key, default=None):
            return next((f['value'] for f in data.get("features", []) if f['key'] == key), default)

        # Extraer características traducidas
        caracteristicas = [
            f"{traducciones.get(f['key'], f['key'].replace('_', ' ').capitalize())}"
            for f in data.get("features", [])
            if f['value'] is not None and f['key'] not in [
                'floor', 
                'surface', 
                'elevator', 
                'furnished',
                'rooms',
                'wardrobes',
                'bathrooms',
                'conservationStatus',
                'antiquity',
                'orientation'
                ]
        ] 
        caracteristicas_str = json.dumps(
            caracteristicas, ensure_ascii=False).replace('"', "'")
        tipo_inmueble = obtener_tipo_inmueble(data.get("buildingType"), data.get("buildingSubtype"))
        calle = data.get('location', '') or ''
        precio = int(data.get("rawPrice", 0) or 0)
        superficie = obtener_caracteristica("surface")
        precio_metro = precio / superficie if precio and superficie else None
        return {
            "id_fotocasa": data.get("id"),
            "tipo": tipo_inmueble,
            "tipo_transaccion": "Alquiler" if data.get("transactionTypeId") == 3 else "Venta",
            "titulo": f"{tipo_inmueble} en {calle}",
            "calle": calle,
            "barrio": data["address"].get("neighborhood") or '',
            "zona": data["address"].get("district") or '',
            "ciudad": data["address"].get("city") or '',
            "localizacion": ", ".join(filter(None, [
                data["address"].get("neighborhood"),
                data["address"].get("district"),
                data["address"].get("city"),
                data["address"].get("county"),
                data["address"].get("regionLevel1")
            ])),
            "precio": precio,
            "precio_metro": precio_metro,
            "caracteristicas": caracteristicas_str,
            "fecha": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "fuente": "fotocasa",
            "disponibilidad": "disponible",
            "superficie": superficie,
            "habitaciones": obtener_caracteristica("rooms"),
            "banos": obtener_caracteristica("bathrooms"),
            "estado": obtener_estado(obtener_caracteristica('conservationStatus')),
            "orientacion": obtener_orientacion(obtener_caracteristica('orientation')),
            "construccion": obtener_antiguedad(obtener_caracteristica('antiquity')),
            "amueblado": "Amueblado" if obtener_caracteristica("furnished") else None,
            "tipo_contacto": obtener_tipo_contacto(data.get("clientTypeId")),
            "nombre_contacto": data.get("clientAlias", 'No indicado') or 'No indicado',
            "telefono_contacto": data.get("phone", "0") or 0,
            "enlace": f"https://www.fotocasa.es{data.get('detail', {}).get('es-ES', '')}"
        }
    except Exception as e:
        print(f"Error extrayendo datos del inmueble: {e}")
        return None

# ==========================
# 🔹 SCRAPING Y EXTRACCIÓN DE DATOS
# ==========================

def obtener_inmuebles(base_url, num_paginas=2):
    session = requests.Session()
    session.headers.update(headers)
    inmuebles = []
    for page in range(1, num_paginas + 1):
        try:
            print(f"Procesando página {page}...")
            response = session.get(f"{base_url}l/{page}?sortType=publicationDate")
            response.raise_for_status()
            json_data = re.search(r'window\.__INITIAL_PROPS__ = JSON\.parse\("((?:[^"\\]|\\.)*)"\);', response.text)
            if json_data:
                json_data = json.loads(json_data.group(1).encode('utf-8').decode('unicode_escape'))
                for inmueble in json_data.get('initialSearch', {}).get('result', {}).get('realEstates', []):
                    info = extract_property_info(inmueble)
                    if info:
                        inmuebles.append(info)
            time.sleep(random.uniform(5, 10))
        except Exception as e:
            print(f"Error en página {page}: {e}")
            break

    return inmuebles

def inmueble_existe(id_fotocasa):
    """
    Verifica si un inmueble con un ID de Fotocasa ya existe en la base de datos a través de la API.

    Args:
        id_fotocasa (int): ID de Fotocasa del inmueble a verificar.

    Returns:
        bool: True si el inmueble existe, False en caso contrario.
    """
    try:
        # Construir la URL para consultar por ID Fotocasa
        url = f"{api_base_url}/inmuebles?id_fotocasa={id_fotocasa}"
        # Realizar la solicitud GET con autenticación
        response = requests.get(url, auth=auth_credentials)
        # Verificar si la solicitud fue exitosa
        if response.status_code == 200:
            data = response.json()
            # Verificar si la API responde con "success" y contiene datos
            if data.get("status") == "success" and data.get("data"):
                return True  # El inmueble existe
            else:
                print(f"⚠️ Inmueble {id_fotocasa} no encontrado en la BD.")
                return False
        else:
            # Manejar otros códigos de estado HTTP
            print(f"⚠️ Error en la API: {response.status_code} - {response.text}")
            return False
    except requests.RequestException as e:
        # Manejar errores de conexión o solicitud
        print(f"🚨 Error al conectar con la API: {e}")
        return False
    
def verificar_insertar_contacto(nombre, telefono, tipo_contacto):
    """
    Verifica si un contacto existe en la base de datos a través de la API. Si no existe, lo crea.
    Si el número de teléfono es 0 o -1, se crea siempre un contacto nuevo.
    
    Args:
        nombre (str): Nombre del contacto.
        telefono (str): Teléfono del contacto.
        tipo_contacto (str): Tipo del contacto.

    Returns:
        int: ID del contacto existente o recién creado, o None en caso de error.
    """
    try:
        # Si el teléfono es 0 o -1, crear siempre un contacto nuevo, ya que no
        # sabemos por el teléfono si ya lo tenemos (Dos Antonios, pero sin teléfono)
        if telefono in ['0', '-1']:
            create_url = f"{api_base_url}/contactos"
            payload = {
                "nombre": nombre,
                "telefono": telefono,
                "tipo_contacto": tipo_contacto
            }
            post_response = requests.post(create_url, auth=auth_credentials, json=payload)

            if post_response.status_code in [200, 201]:
                created_data = post_response.json()
                if created_data.get("status") == "success" and created_data.get("data"):
                    return created_data["data"]["id"]  # Devuelve el ID del contacto recién creado
                else:
                    print(f"Error al procesar la creación: {created_data.get('message', 'Mensaje no disponible')}")
                    return None
            else:
                print(f"Error al crear el contacto: {post_response.status_code} - {post_response.text}")
                return None

        # Construir la URL para buscar contactos por nombre y teléfono
        url = f"{api_base_url}/contactos"
        params = {"nombre": nombre, "telefono": telefono}

        # Realizar la solicitud GET para buscar el contacto
        response = requests.get(url, auth=auth_credentials, params=params)

        # Manejo de la respuesta GET
        if response.status_code == 200:
            data = response.json()

            if data.get("status") == "success":
                if data.get("data"):  # Si 'data' no está vacío, el contacto existe
                    return data["data"][0]["id_contacto"]  # Devuelve el ID del primer contacto encontrado
                else:
                    # Si el 'data' está vacío, proceder a crear el contacto
                    print("No se encontró ningún contacto con los filtros dados. Procediendo a crear el contacto.")
            else:
                print(f"Error en la búsqueda: {data.get('message', 'Mensaje no disponible')}")
                return None
        elif response.status_code == 404:
            # Si el código es 404, el contacto no existe, proceder a crearlo
            print("El contacto no existe. Procediendo a crear el contacto.")
        else:
            # Manejar códigos de estado inesperados
            print(f"Error inesperado al buscar el contacto: {response.status_code} - {response.text}")
            return None

        # Si llegamos aquí, significa que el contacto no existe. Procedemos a crearlo.
        create_url = f"{api_base_url}/contactos"
        payload = {
            "nombre": nombre,
            "telefono": telefono,
            "tipo_contacto": tipo_contacto
        }

        # Realizar la solicitud POST para crear el contacto
        post_response = requests.post(create_url, auth=auth_credentials, json=payload)

        if post_response.status_code in [200, 201]:
            created_data = post_response.json()
            if created_data.get("status") == "success" and created_data.get("data"):
                return created_data["data"]["id"]  # Devuelve el ID del contacto recién creado
            else:
                print(f"Error al procesar la creación: {created_data.get('message', 'Mensaje no disponible')}")
                return None
        else:
            print(f"Error al crear el contacto: {post_response.status_code} - {post_response.text}")
            return None

    except requests.RequestException as e:
        print(f"Error al conectar con la API: {e}")
        return None

def insertar_inmueble(data):
    """
    Inserta un inmueble en la base de datos a través de la API.
    Args:
        data (dict): Diccionario con los datos del inmueble.
    Returns:
        int: ID del inmueble recién creado, o None en caso de error.
    """
    # Filtrar solo los campos especificados
    allowed_keys = [
        "id_fotocasa", "id_contacto", "tipo", "titulo", "calle", "barrio", "zona", "ciudad", "localizacion", "precio",
        "precio_metro", "superficie", "habitaciones", "banos", "armarios", "trastero", "orientacion", "amueblado",
        "calefaccion", "planta", "ascensor", "construccion", "movilidad_reducida", "exterior_interior", "fecha",
        "estado", "caracteristicas", "fuente", "disponibilidad", "tipo_transaccion", "enlace"
    ]
    filtered_data = {key: value for key, value in data.items() if key in allowed_keys}
    try:
        # Construir la URL para insertar un inmueble
        url = f"{api_base_url}/inmuebles"
        # Realizar la solicitud POST con los datos del inmueble
        response = requests.post(url, auth=auth_credentials, json=filtered_data)
        # Verificar si la inserción fue exitosa
        if response.status_code in [200, 201]:
            created_data = response.json()
            if created_data.get("status") == "success":
                # Obtener el ID del inmueble creado desde 'data'
                inmueble_id = created_data.get("data", {}).get("id")
                if inmueble_id:
                    print(f"ID {inmueble_id} guardado en la base de datos.")
                    return inmueble_id  # Devuelve el ID del inmueble recién creado
                else:
                    print("No se pudo obtener el ID del inmueble creado.")
                    return None
            else:
                print(f"Error en la respuesta: {created_data.get('message', 'Mensaje no disponible')}")
                return None
        else:
            print(f"Error al insertar el inmueble: {response.status_code} - {response.text}")
            return None
    except requests.RequestException as e:
        print(f"Error al conectar con la API: {e}")
        return None

def procesar_inmuebles(base_url, max_repetidos=5, num_paginas=10):
    """
    Procesa los inmuebles desde Fotocasa y los inserta en la base de datos.
    Detiene el proceso si se encuentran demasiados repetidos seguidos.

    Args:
        base_url (str): URL base de Fotocasa.
        max_repetidos (int): Número máximo de inmuebles consecutivos repetidos antes de detener el proceso.
        num_paginas (int): Número máximo de páginas a procesar.
    """
    session = requests.Session()
    session.headers.update(headers)
    repetidos_consecutivos = 0  # Contador de repetidos seguidos
    inmuebles_insertados = 0  # Contador de inmuebles insertados correctamente

    for page in range(1, num_paginas + 1):
        try:
            print(f"Procesando página {page}...")
            response = session.get(f"{base_url}l/{page}?sortType=publicationDate")
            
            if response.status_code in [403, 429]:
                print(f"⚠️ Acceso bloqueado ({response.status_code}). Deteniendo scraping.")
                break

            response.raise_for_status()  # Lanza error si la request falla

            # Extraer JSON de la página
            json_data = re.search(r'window\.__INITIAL_PROPS__ = JSON\.parse\("((?:[^"\\]|\\.)*)"\);', response.text)
            if json_data:
                json_data = json.loads(json_data.group(1).encode('utf-8').decode('unicode_escape'))
                inmuebles = json_data.get('initialSearch', {}).get('result', {}).get('realEstates', [])

                for inmueble_data in inmuebles:
                    id_fotocasa = inmueble_data.get("id")
                    # 🔍 **Verificar si ya existe en la base de datos**
                    if inmueble_existe(id_fotocasa):
                        repetidos_consecutivos += 1
                        print(f"🔁 Inmueble {id_fotocasa} ya en la BD. Contador: {repetidos_consecutivos}")
                        if repetidos_consecutivos >= max_repetidos:
                            print("🚨 Máximo de repetidos alcanzado. Deteniendo scraping.")
                            return
                        continue  # Salta al siguiente inmueble
                    else:
                        repetidos_consecutivos = 0  # Reiniciar contador si encontramos un nuevo inmueble
                    # Extraer y limpiar datos del inmueble
                    inmueble_info = extract_property_info(inmueble_data)
                    if inmueble_info:
                        # 🏡 **Gestión del contacto**
                        id_contacto = verificar_insertar_contacto(
                            inmueble_info.get("nombre_contacto"),
                            str(inmueble_info.get("telefono_contacto")),
                            str(inmueble_info.get("tipo_contacto"))
                        )
                        if id_contacto:
                            inmueble_info['id_contacto'] = id_contacto
                            insertar_inmueble(inmueble_info)
                        # **Insertar el inmueble en la BD**
                        inmuebles_insertados += 1
                        print(f"✅ Inmueble {id_fotocasa} insertado correctamente.")
            # Espera aleatoria para evitar bloqueos
            time.sleep(random.uniform(5, 10))
        except requests.RequestException as e:
            print(f"⚠️ Error en página {page}: {e}")
            break  # Detener el scraping si hay errores de red graves
    print(f"🎯 Proceso completado. {inmuebles_insertados} inmuebles insertados en la base de datos.")

procesar_inmuebles('https://www.fotocasa.es/es/alquiler/viviendas/murcia-provincia/todas-las-zonas/', 5, 1)
# print(obtener_inmuebles('https://www.fotocasa.es/es/alquiler/viviendas/murcia-provincia/todas-las-zonas/',1))


Procesando página 1...
⚠️ Inmueble 185513687 no encontrado en la BD.
ID 342343231 guardado en la base de datos.
✅ Inmueble 185513687 insertado correctamente.
⚠️ Inmueble 183656135 no encontrado en la BD.
ID 342343232 guardado en la base de datos.
✅ Inmueble 183656135 insertado correctamente.
⚠️ Inmueble 185496670 no encontrado en la BD.
No se encontró ningún contacto con los filtros dados. Procediendo a crear el contacto.
ID 342343233 guardado en la base de datos.
✅ Inmueble 185496670 insertado correctamente.
⚠️ Inmueble 185510967 no encontrado en la BD.
No se encontró ningún contacto con los filtros dados. Procediendo a crear el contacto.
ID 342343234 guardado en la base de datos.
✅ Inmueble 185510967 insertado correctamente.
⚠️ Inmueble 185510130 no encontrado en la BD.


KeyboardInterrupt: 